# WZK - Zadanie 2

Autor: Dariusz Max Adamski - nr indeksu 136674

---

In [2]:
import numpy as np
from random import randint
from math import gcd

## 1. Szybkie sprawdzanie liczb pierwszych

Do rozwiązania zadan jest potrzebny generator dużych liczb pierwszych.

W tym celu zaimplementowałem algorytm Millera-Rabina do szybkiego sprawdzenia czy liczba (prawdopodobnie) jest liczbą pierwszą. Dla $k$ iteracji algorytmu, prawdopodobieństwo popełnienia błędu jest równe co najwyżej $4^{-k}$. Jako $k$ wybrałem moją ulubioną liczbę 42, co daje prawdopodobieństwo pomyłki równe około $5.17 \cdot 10^{-26}$. Bardziej prawdopodobne niż pomyłka tego algorytmu, jest to, że losowo wybierając gwiazdę we wszechświecie akurat wybierzemy nasze Słońce!

In [7]:
def is_witness(w, n, r, d):
    w = pow(w, d, n)
    if w == 1 or w == n - 1:
        return False
    for _ in range(r):
        w = pow(w, 2, n)
        if w == n - 1:
            return False
    return True

def is_prime(n, rounds=42):
    if n <= 1: return False
    if n <= 3: return True
    
    # n = 2^r * d + 1
    r, d = 0, n - 1
    while d % 2 == 0:
        d //= 2
        r += 1
        
    for _ in range(rounds):
        w = randint(2, n - 2)
        if is_witness(w, n, r, d):
            return False
    return True

## 2. Algorytm Blum-Blum-Shub

W algorytmie BBS na początku losuję dwie duże liczby pierwsze $p$ i $q$, które są przystające do 3 modulo 4.
Losuję też liczbę $x$ taką, że $pq$ i $x$ są względnie pierwsze, co jest równoznaczne z tym, że ich największy wspólny dzielnik jest równy 1. Następnie tyle razy ile chcę pozyskać bitów, ustawiam $x := x^2\ \text{mod}\ n$ i dodaję namniej znaczący bit do wynikowego ciągu.

In [40]:
def random_where(lb, ub, where):
    while True:
        n = randint(lb, ub)
        if where(n): return n
        
def random_bbs(n_bits, p=None, q=None, x0=None):
    lb, ub = 2**32, 2**40
    p = p or random_where(lb, ub, where=lambda x: x % 4 == 3 and is_prime(x))
    q = q or random_where(lb, ub, where=lambda x: x % 4 == 3 and is_prime(x))
    n = p * q
    x0 = x0 or random_where(0, 2**32, where=lambda x: gcd(n, x) == 1)
    assert gcd(n, x0) == 1
    x = x0
    r = 0
    for _ in range(n_bits):
        x = pow(x, 2, n)
        r = (r << 1) | (x & 1)
        print(x & 1, end='')
    print()
        
    # demonstration 
    assert p % 4 == 3 and q % 4 == 3 and gcd(n, x) == 1
    print(f'p  = {p}\nq  = {q}\nn  = {n}\nx0 = {x0}')
    print(f'r  = {str(r)[:50]}')
    
    return r
    
r = random_bbs(p=7, q=11, x0=12, n_bits=100)

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
p  = 7
q  = 11
n  = 77
x0 = 12
r  = 1267650600228229401496703205375


## 3. Testy losowości

Aby przetestować losowość wygenerowanego ciągu należy użyć testów statystycznych. Zestaw prymitywnych testów jest zawarty w standardzie FIPS 140-2. Poniżej umieszczone są implementacje czterech testów losowości z tego standardu.

In [5]:
def monobit_test(x):
    n = 0
    for _ in range(20_000):
        b = x & 1
        x = x >> 1
        if b == 1:
            n += 1
    return 9725 < n < 10275

def runs_test(x):
    runs = np.zeros(6)
    last = x & 1
    x = x >> 1
    n = 1
    for _ in range(20_000):
        b = x & 1
        x = x >> 1
        if b != 0:
            runs[min(n, 6) - 1] += 1
            last = b
            n = 0
        n += 1
    s1, s2, s3, s4, s5, s6 = runs // 2
    return (2315 <= s1 <= 2685 and
            1114 <= s2 <= 1386 and
            527  <= s3 <= 723 and
            240  <= s4 <= 384 and
            103  <= s5 <= 209 and
            103  <= s6 <= 209)

def long_run_test(x):
    last = None
    max_n = 0
    n = 0
    for _ in range(20_000):
        b = x & 1
        x = x >> 1
        if b != last:
            last = b
            max_n = max(n, max_n)
            n = 0
        n += 1
    return max_n < 26

def poker_test(x):
    count = np.zeros(shape=16)
    a, b, c, d = x >> 0, x >> 5000, x >> 10000, x >> 15000
    for _ in range(5000):
        i = ((a&1) << 0) | ((b&1) << 1) | ((c&1) << 2) | ((d&1) << 3)
        count[i] += 1
        a, b, c, d = a >> 1, b >> 1, c >> 1, d >> 1
    criterion = 16/5000 * np.sum(count**2) - 5000
    return 2.16 < criterion < 46.17

def all_tests(x):
    names = ['monobit', 'runs', 'long run', 'poker']
    tests = [monobit_test, runs_test, long_run_test, poker_test]
    for name, f in zip(names, tests):
        result = 'passed' if f(x) else 'failed'
        print(f'{name} test\t{result}')
        
all_tests(r)

monobit test	passed
runs test	passed
long run test	passed
poker test	passed


Wygenerowany ciąg przeszedł wszystkie testy FIPS 140-2. Nie oznacza to jednak, że ciąg generowany przez ten algorytm jest nieprzewidywalny. Wykazano, że testy losowości FIPS 140-2 mogą zostać oszukane przez ciągi generowane z intencjonalnie wadliwych generatorów losowych ciągów (poniższa publikacja). Sam standard FIPS 140-2 został zastąpiony nowszym FIPS 140-3 w 2019 roku, a w 2026 roku testy z FIPS 140-2 zostaną oznaczone jako historyczne.

D. Hurley-Smith, C. Patsakis and J. Hernandez-Castro, "On the unbearable lightness of FIPS 140-2 randomness tests," in IEEE Transactions on Information Forensics and Security, doi: 10.1109/TIFS.2020.2988505.

## 4. Dodatkowe testy

Sprawdziłem też ciągi generowane z urządzenia systemowego `/dev/random` i wadliwego generatora zwracającego ciąg z naprzemiennymi zerami i jedynkami. Tak jak się można było spodziewać, pierwszy przechodzi wszystkie testy, a drugi nie.

In [6]:
import os
ref = int.from_bytes(os.urandom(20_000), 'little')
print('-- /dev/random --')
all_tests(ref)

-- /dev/random --
monobit test	passed
runs test	passed
long run test	passed
poker test	passed


In [7]:
print('-- 0101010... --')
bad = 0
last = 0
for _ in range(20_000):
    bad = (bad << 1) | (last & 1)
    last = 1 - last
all_tests(bad)

-- 0101010... --
monobit test	passed
runs test	failed
long run test	passed
poker test	failed
